In [ ]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern

def analyze_drain_balanced_tuning(image_path):
    # 1) HSV/BGR thresholds (흙/모래와 격자 오탐의 균형점 찾기)
    # 갈색: V 하한을 5로 유지 (너무 어둡지 않은 갈색 흙 포착)
    brown_lower = np.array([0,  20,  5])
    brown_upper = np.array([70, 255, 255])
    
    # 회색: S 15, V 20으로 조정하여 어두운 흙/모래를 일부 포착하면서도 격자 제외 시도
    gray_lower  = np.array([0,   15,  20])   # S 0->15, V 0->20
    gray_upper  = np.array([180,100,200])    # V 상한 유지

    pack_lower  = np.array([0,   80,  50])
    pack_upper  = np.array([180,255,255])
    
    green_lower = np.array([30, 20, 10])
    green_upper = np.array([90, 255, 255])

    white_lower = np.array([150,150,150])   # 흰색 (유지)
    white_upper = np.array([255,255,255])

    # 2) Load ROI image
    roi = cv2.imread(image_path)
    if roi is None:
        print(f"Cannot load image: {image_path}")
        return None

    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

    # 3) Build masks (기존 방식)
    mask_brown = cv2.inRange(hsv, brown_lower, brown_upper)
    mask_gray  = cv2.inRange(hsv, gray_lower,  gray_upper)
    soil_mask  = cv2.bitwise_or(mask_brown, mask_gray)
    soil_mask = cv2.morphologyEx(soil_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8), iterations=1)
    soil_mask = cv2.morphologyEx(soil_mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8), iterations=1)

    pack_mask  = cv2.inRange(hsv, pack_lower, pack_upper)
    pack_mask  = cv2.bitwise_and(pack_mask, cv2.bitwise_not(soil_mask)) 
    pack_mask = cv2.morphologyEx(pack_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8), iterations=1)

    green_mask = cv2.inRange(hsv, green_lower, green_upper)
    green_mask = cv2.morphologyEx(green_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8), iterations=1)

    white_mask_raw  = cv2.inRange(roi, white_lower, white_upper) 
    cig_white_mask = cv2.morphologyEx(white_mask_raw,
                                   cv2.MORPH_OPEN,
                                   np.ones((3,3), np.uint8))
    cig_white_mask = cv2.morphologyEx(cig_white_mask, 
                                   cv2.MORPH_CLOSE,
                                   np.ones((5,5), np.uint8), iterations=1)

    # --- 질감(LBP) 특징 마스크 (격자 제외를 위해 다시 조금 높임) ---
    lbp_image = local_binary_pattern(gray, P=8, R=1, method='uniform')
    
    # LBP 임계값: 0에서 20으로 다시 올림 (격자 오탐 줄이고자, 흙/모래 일부 손실 감수)
    lbp_threshold = 20 
    lbp_mask = (lbp_image > lbp_threshold).astype(np.uint8) * 255
    lbp_mask = cv2.morphologyEx(lbp_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8), iterations=1)
    lbp_mask = cv2.morphologyEx(lbp_mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8), iterations=1)


    # --- 모든 마스크 결합 ---
    combined_initial_mask = cv2.bitwise_or(soil_mask, pack_mask)
    combined_initial_mask = cv2.bitwise_or(combined_initial_mask, green_mask)
    combined_initial_mask = cv2.bitwise_or(combined_initial_mask, cig_white_mask)
    combined_initial_mask = cv2.bitwise_or(combined_initial_mask, lbp_mask) 

    # --- 형태학적 필터링 (쓰레기 영역 채우기) ---
    kernel_small = np.ones((3,3), np.uint8)
    kernel_medium = np.ones((5,5), np.uint8)
    
    processed_mask = cv2.morphologyEx(combined_initial_mask, cv2.MORPH_OPEN, kernel_small, iterations=1)
    processed_mask = cv2.morphologyEx(processed_mask, cv2.MORPH_CLOSE, kernel_medium, iterations=2) 
    processed_mask = cv2.morphologyEx(processed_mask, cv2.MORPH_OPEN, kernel_small, iterations=1)
    
    final_contamination_mask = processed_mask

    # --- 격자 제외 로직 (격자 형태 감지 파라미터는 이전 값으로 유지) ---
    # Canny 임계값도 격자 형태에 맞춰 튜닝된 값 유지
    blurred_gray = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(blurred_gray, 20, 60) 

    # 디버깅용: Canny 엣지 시각화
    # cv2.imshow("Canny Edges (Grid)", edges)
    # cv2.waitKey(0)

    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    grid_mask = np.zeros_like(gray, dtype=np.uint8)
    image_area = roi.shape[0] * roi.shape[1]
    
    for c in contours:
        area = cv2.contourArea(c)
        if area < 100 or area > 1500: # 격자 칸 면적 범위는 이미지에 따라 튜닝
            continue
        
        epsilon = 0.03 * cv2.arcLength(c, True) 
        approx = cv2.approxPolyDP(c, epsilon, True)

        if len(approx) >= 4 and len(approx) <= 5: 
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = float(w)/h
            
            if aspect_ratio >= 0.7 and aspect_ratio <= 1.3: 
                cv2.rectangle(grid_mask, (x,y), (x+w, y+h), 255, cv2.FILLED)

    # 격자 마스크 자체 정제 (강화된 설정 유지)
    grid_mask = cv2.morphologyEx(grid_mask, cv2.MORPH_CLOSE, np.ones((12,12), np.uint8), iterations=2)
    grid_mask = cv2.morphologyEx(grid_mask, cv2.MORPH_OPEN, np.ones((9,9), np.uint8), iterations=1)

    # 디버깅용: 최종 Grid Mask 시각화
    # cv2.imshow("Final Grid Mask", grid_mask)
    # cv2.waitKey(0)

    # 5. 최종 오염 마스크에서 격자 마스크 제외 (가장 중요)
    final_contamination_mask = cv2.bitwise_and(final_contamination_mask, cv2.bitwise_not(grid_mask))

    # 4) Compute combined ratio
    h_, w_ = roi.shape[:2]
    total_pixels = h_ * w_
    contaminated_pixels = cv2.countNonZero(final_contamination_mask)
    
    if total_pixels > 0:
        combined_ratio = (contaminated_pixels / total_pixels) * 100
    else:
        combined_ratio = 0.0

    print(f"Combined contamination ratio: {combined_ratio:.1f}%")

    # --- 결과 시각화 ---
    overlay = roi.copy()
    color_mask = np.zeros_like(roi, dtype=np.uint8)
    color_mask[final_contamination_mask == 255] = [0, 0, 255] # BGR (빨간색)

    alpha = 0.5 # 투명도 설정
    result_image = cv2.addWeighted(overlay, 1 - alpha, color_mask, alpha, 0)
    
    # 이미지 크기 조절
    display_width = 800 
    display_height = int(roi.shape[0] * (display_width / roi.shape[1]))
    
    resized_original = cv2.resize(roi, (display_width, display_height), interpolation=cv2.INTER_AREA)
    resized_result = cv2.resize(result_image, (display_width, display_height), interpolation=cv2.INTER_AREA)

    combined_display_image = np.hstack((resized_original, resized_result))

    cv2.imshow(f"Original vs Contamination ({combined_ratio:.1f}%)", combined_display_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return combined_ratio

# 이미지 파일 경로 설정
image_path = "C:/Users/USER/Pictures/Screenshots/2025-06-09 172304.png"

# 함수 호출
contamination_percentage = analyze_drain_balanced_tuning(image_path)
if contamination_percentage is not None:
    print(f"{image_path}의 오염 영역은 약 {contamination_percentage:.1f}% 입니다.")

Combined contamination ratio: 93.7%
C:/Users/USER/Pictures/Screenshots/2025-06-09 172304.png의 오염 영역은 약 93.7% 입니다.
